# Importamos librerias

In [45]:
import pandas as pd
import snowflake.connector
import json
from querys import *
from functions import *
from datetime import datetime
pd.options.mode.chained_assignment = None

# SCOPES

In [37]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
DELETE_SCOPES = ['https://www.googleapis.com/auth/drive.file']

# Folder ID

In [48]:
# Ya sea que el codigo se ejecute desde mi pc o desde la VM, el folder ID sera siempre el mismo
# ya que siempre guardaremos los spreadsheets en la misma carpeta

FOLDER_ID = '11yV-IyMUXCjQRvXBiDNYIEg_pDc6HglL' # Mi Unidad / Promos
#FOLDER_ID = '1pMhtpNJCi_sJ4xxXmRFC4YkIrzOYtZX1' # Mi Unidad / Otros

# if os.getcwd() == 'C:\\Users\\leonardo.mangold\\PycharmProjects\\promos_inteligencia_negocio':
#     FOLDER_ID = '1VCclV4q67yME8f0Guy2jKD5HeAQLgkUz'
# else:
#     FOLDER_ID = '1js40bDjRG7XcxeDUGoWYy4wBbsDPjwMu'


# Drive Leo 11yV-IyMUXCjQRvXBiDNYIEg_pDc6HglL
# Drive Unidad Compartida Inteligencia de Negocio 1VCclV4q67yME8f0Guy2jKD5HeAQLgkUz

FOLDER_ID

'11yV-IyMUXCjQRvXBiDNYIEg_pDc6HglL'

# Spreadsheet ID

In [53]:
# Mi Unidad / Otros

url = 'https://docs.google.com/spreadsheets/u/1/d/1FkdwD5G7c0EOAzFaUAYmDsCAi8jzcZcO/edit?usp=drive_web&ouid=113624480832701744876&rtpof=true'

# Mi Unidad / Promos

# url = 'https://docs.google.com/spreadsheets/d/1w5oGifgavq1GodX3z9t9UF8ALeOl6NzMWiu_4xPG8FE/edit'

# Extracting spreadsheet ID

spreadsheet_id = url.split('/')[-2]

spreadsheet_id

'1FkdwD5G7c0EOAzFaUAYmDsCAi8jzcZcO'

In [57]:
credentials = get_credentials()

# Authenticate with Google Sheets API using service account credentials
gc = gspread.authorize(credentials)

# Open the spreadsheet by its ID
spreadsheet = gc.open_by_key(spreadsheet_id)

APIError: {'code': 400, 'message': 'This operation is not supported for this document', 'status': 'FAILED_PRECONDITION'}

In [58]:
spreadsheet

<Spreadsheet 'Promos' id:1w5oGifgavq1GodX3z9t9UF8ALeOl6NzMWiu_4xPG8FE>

# Conexion Snowflake

In [20]:
# Conexion Snowflake desde compu personal

if os.getcwd() == 'C:\\Users\\leonardo.mangold\\PycharmProjects\\promos_inteligencia_negocio':

	while True:
	    try:

	        f = open('credentials.json')
	        data_pass = json.load(f)

	        pass_ = input("INGRESAR PASSCODE:")

	        ctx = snowflake.connector.connect(
	            user = data_pass['snow']['USER'],
	            password = data_pass['snow']['PASS'],
	            account = data_pass['snow']['ACCOUNT'],
	            passcode = pass_,
	            database = 'SANDBOX_PLUS',
	            schema = 'DWH'
	        )

	        cursor = ctx.cursor()

	        print('Connected')

	        break

	    except:
	        print('Incorrect Password - provide again')

# Conexion VM a Snowflake

else:

    ctx = snowflake.connector.connect(
        user="PLUS_VM1",
        password="Plus22!",
        account="XZ23267-dp32414",
        database="SANDBOX_PLUS",
        schema="DWH"
        )

    cursor = ctx.cursor()

Connected


# Fechas indicadas por Juli para tomar los precios

In [21]:
url_fechas = 'https://docs.google.com/spreadsheets/d/1JnayuiljvaOebik4gKqNgxSLD6RO2AS8euG7apQykW8/export?format=csv'
fechas = pd.read_csv(url_fechas)
fechas = fechas[pd.to_datetime(fechas['HASTA']) + pd.Timedelta(days=1) >= datetime.today()]

# 1. Precios Oferta

In [22]:
df_precios_oferta = pd.DataFrame()
for i in range(len(fechas)):
    desde = fechas.iloc[i]['DESDE']
    hasta = fechas.iloc[i]['HASTA']
    oferta = fechas.iloc[i]['OFERTA']

    cursor.execute(precios_oferta.format(desde_snow = desde))
    df_aux = cursor.fetch_pandas_all()
    df_aux['PRECIO DESDE'] = desde
    df_aux['PRECIO HASTA'] = hasta
    df_aux['NOMBRE_PROMO'] = oferta
    df_precios_oferta = pd.concat([df_precios_oferta, df_aux])

df_precios_oferta = df_precios_oferta[['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN', 'NOMBRE_PROMO', 'PRECIO DESDE', 'PRECIO HASTA', 'ORIN', 'ESTADISTICO', 'PROM_PVP_OFERTA']]

df_precios_oferta['FIN'] = pd.to_datetime(df_precios_oferta['FIN'])

df_precios_oferta.sort_values(by = ['PRECIO HASTA', 'NOMBRE_PROMO', 'FIN'], ascending = [False, True, False], inplace = True)
df_precios_oferta['FIN'] = df_precios_oferta['FIN'].astype(str)
df_precios_oferta['INICIO'] = df_precios_oferta['INICIO'].astype(str)
df_precios_oferta['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')

# 2. Precios Stock Mediano dia de ayer

In [23]:
cursor.execute(precios_stock_mediano)
df_precios_stock_mediano = cursor.fetch_pandas_all()
df_precios_stock_mediano['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')

# 3. OPT

In [24]:
cursor.execute(opt)
df_opt = cursor.fetch_pandas_all()
df_opt['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')

# 4. Locales Activos Ayer

In [25]:
cursor.execute(locales_activos_ayer)
df_locales_activos_ayer = cursor.fetch_pandas_all()
df_locales_activos_ayer['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')

# 5. Days on Hand

## Parte A - Articulos

In [46]:
cursor.execute(days_on_hand_articulo)
df_days_on_hand_articulo = cursor.fetch_pandas_all()

df_days_on_hand_articulo = df_days_on_hand_articulo[df_days_on_hand_articulo['UNIDADES'] != 0]
df_days_on_hand_articulo['DAYS ON HAND'] = df_days_on_hand_articulo['UNIDADES'] / df_days_on_hand_articulo['UNIDADES_VENDIDAS']
df_days_on_hand_articulo['DAYS ON HAND'][(df_days_on_hand_articulo['UNIDADES'] == 0) & (df_days_on_hand_articulo['UNIDADES_VENDIDAS'] == 0)] = 0
df_days_on_hand_articulo['DAYS ON HAND'][df_days_on_hand_articulo['DAYS ON HAND'] == np.inf] = 999999
df_days_on_hand_articulo['DAYS ON HAND'][df_days_on_hand_articulo['DAYS ON HAND'] == -np.inf] = -999999
df_days_on_hand_articulo['DAYS ON HAND'][df_days_on_hand_articulo['DAYS ON HAND'] > 999999] = 999999
df_days_on_hand_articulo['DAYS ON HAND'][df_days_on_hand_articulo['DAYS ON HAND'] < -999999] = -999999
df_days_on_hand_articulo['DAYS ON HAND'].fillna(999999, inplace=True)
df_days_on_hand_articulo['DAYS ON HAND'] = round(df_days_on_hand_articulo['DAYS ON HAND'], 0).astype(int)
df_days_on_hand_articulo['UNIDADES'] = round(df_days_on_hand_articulo['UNIDADES'], 0).astype(int)
df_days_on_hand_articulo['UNIDADES_VENDIDAS'] = round(df_days_on_hand_articulo['UNIDADES_VENDIDAS'], 0).astype(int)
df_days_on_hand_articulo.rename({'UNIDADES_VENDIDAS':'UNIDADES VENDIDAS'}, axis = 1, inplace = True)
df_days_on_hand_articulo = df_days_on_hand_articulo[['SUBCLASE', 'ORIN', 'UNIDADES', 'UNIDADES VENDIDAS', 'DAYS ON HAND']]

df_days_on_hand_articulo.rename({'DAYS ON HAND':'DAYS ON HAND ARTICULO'}, axis = 1, inplace = True)

## Parte B - Subclases

In [47]:
cursor.execute(days_on_hand_subclase)
df_days_on_hand_subclase = cursor.fetch_pandas_all()

df_days_on_hand_subclase = df_days_on_hand_subclase[df_days_on_hand_subclase['UNIDADES'] != 0]
df_days_on_hand_subclase['DAYS ON HAND'] = df_days_on_hand_subclase['UNIDADES'] / df_days_on_hand_subclase['UNIDADES_VENDIDAS']
df_days_on_hand_subclase['DAYS ON HAND'][(df_days_on_hand_subclase['UNIDADES'] == 0) & (df_days_on_hand_subclase['UNIDADES_VENDIDAS'] == 0)] = 0
df_days_on_hand_subclase['DAYS ON HAND'][df_days_on_hand_subclase['DAYS ON HAND'] == np.inf] = 999999
df_days_on_hand_subclase['DAYS ON HAND'][df_days_on_hand_subclase['DAYS ON HAND'] == -np.inf] = -999999
df_days_on_hand_subclase['DAYS ON HAND'][df_days_on_hand_subclase['DAYS ON HAND'] > 999999] = 999999
df_days_on_hand_subclase['DAYS ON HAND'][df_days_on_hand_subclase['DAYS ON HAND'] < -999999] = -999999
df_days_on_hand_subclase['DAYS ON HAND'].fillna(999999, inplace=True)
df_days_on_hand_subclase['DAYS ON HAND'] = round(df_days_on_hand_subclase['DAYS ON HAND'], 0).astype(int)
df_days_on_hand_subclase['UNIDADES'] = round(df_days_on_hand_subclase['UNIDADES'], 0).astype(int)
df_days_on_hand_subclase['UNIDADES_VENDIDAS'] = round(df_days_on_hand_subclase['UNIDADES_VENDIDAS'], 0).astype(int)
df_days_on_hand_subclase.rename({'UNIDADES_VENDIDAS':'UNIDADES VENDIDAS'}, axis = 1, inplace = True)
df_days_on_hand_subclase = df_days_on_hand_subclase[['SUBCLASE', 'UNIDADES', 'UNIDADES VENDIDAS', 'DAYS ON HAND']]

df_days_on_hand_subclase.rename({'DAYS ON HAND':'DAYS ON HAND SUBCLASE'}, axis = 1, inplace = True)

    # Parte C - Consolido en Articulos

df_days_on_hand_articulo = df_days_on_hand_articulo.merge(df_days_on_hand_subclase[['SUBCLASE', 'DAYS ON HAND SUBCLASE']], on = 'SUBCLASE', how = 'left')
df_days_on_hand_articulo['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')

# Corremos main

In [50]:
credentials = get_credentials()

# Debido a que siempre mantenemos la misma spreadsheet, su id no cambia

# Create a Google Sheets API service --> esto se usa en caso de usar la funcion 2
service = build('sheets', 'v4', credentials=credentials)

# Create sheets and insert the DataFrames

dataframes_dict = {
    'df_precios_oferta': 'Precios Oferta',
    'df_precios_stock_mediano': 'Precios Stock Mediano',
    'df_opt': 'OPT',
    'df_locales_activos_ayer': 'Locales Activos Ayer',
    'df_days_on_hand_articulo': 'Days on Hand - Articulos'
}

for df_name, sheet_name in dataframes_dict.items():

    # Elimino la primera sheet
    delete_first_sheet(spreadsheet_id, credentials)

    df = globals()[df_name]  # Retrieve the dataframe using its name
    print(df_name, sheet_name)

    # Inserto la DataFrame en una nueva sheet, que ocupa la ultima posicion
    insert_dataframe_into_sheet(df, spreadsheet_id, credentials, sheet_name)


print(f"Lineas df_precios_oferta --> {df_precios_oferta.shape[0]}")

df_precios_oferta Precios Oferta
df_precios_stock_mediano Precios Stock Mediano
df_opt OPT
df_locales_activos_ayer Locales Activos Ayer
df_days_on_hand_articulo Days on Hand - Articulos
Lineas df_precios_oferta --> 14029


# Fin